0. **Only keep French tweets**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Only keep French tweets
!pip install -q langdetect
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 42  # Make results deterministic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
def is_french_langdetect(text):
    try:
        return detect(str(text)) == 'fr'
    except:
        return False  # Skip unrecognizable text

In [ ]:
import pandas as pd

# Set file path
file_path = '/content/drive/My Drive/_thesis_dogwhistles/Clean-tweets/new_left_10462.csv'

# Load CSV and ensure 'id' is string
df = pd.read_csv(file_path, dtype={'id': str})
df = df.dropna(subset=['content'])

# Count before
total_before = len(df)

# Filter French tweets using langdetect
df_fr = df[df['content'].apply(is_french_langdetect)].reset_index(drop=True)

# Count after
total_after = len(df_fr)
removed = total_before - total_after
percentage_removed = (removed / total_before) * 100

# Show summary
print(f"Total tweets before filtering: {total_before}")
print(f"Total tweets after filtering: {total_after}")
print(f"Tweets removed: {removed} ({percentage_removed:.2f}%)")

# Show sample
if len(df_fr) > 0:
    print("\nSample of 5 French tweets:")
    print(df_fr['content'].sample(5, random_state=42).to_string(index=False))
else:
    print("⚠️ No French tweets detected.")

# Save result
output_path = '/content/drive/My Drive/_thesis_dogwhistles/Clean-tweets/left-new-french-tweets.csv'
df_fr.to_csv(output_path, index=False)
print(f"\n✅ French tweets saved to: {output_path}")

Total tweets before filtering: 100635
Total tweets after filtering: 73793
Tweets removed: 26842 (26.67%)

Sample of 5 French tweets:
         @Perrette56 Merci beaucoup c'est gentil 😘
Adios le "spécialiste des penaltys" ...🤡\n#Merc...
@ArSaint_Martin @EmilieHaon Un grand nombre de ...
    @RemikenzoPages tout ça c’est grâce à l’AJAR😝🩷
             #QuelleEpoque https://t.co/kaV5QSKBYS

✅ French tweets saved to: /content/drive/My Drive/_thesis_dogwhistles/Clean-tweets/left-new-french-tweets.csv


1. Define function


* Remove URLs
* Remove @usernames
* Remove leading #
* Normalize whitespace
* Lowercase
* Save clean CSV

In [ ]:
import pandas as pd
import re

def clean_text(text):
    text = str(text)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)          # Remove URLs
    text = re.sub(r'@\w+', '', text)                             # Remove @usernames
    text = re.sub(r'#(\w+)', r'\1', text)                        # Remove leading #
    text = re.sub(r'\s+', ' ', text).strip()                     # Normalize whitespace
    text = text.lower()                                          # Lowercase text

    return text

def clean_tweets(input_path, output_path):
    # Load CSV
    df = pd.read_csv(input_path, dtype={'id': str})

    # Remove rows where 'content' is missing
    df = df.dropna(subset=['content'])

    # Apply cleaning function
    df['content'] = df['content'].apply(clean_text)

    # Remove empty content rows again
    df = df[df['content'].str.strip() != '']

    # Output info
    print(f"Remaining rows: {len(df)}")
    total_tokens = df['content'].str.split().str.len().sum()
    print(f"Total number of tokens: {total_tokens}")

    # Save cleaned file
    df.to_csv(output_path, index=False)

    return df

2. Call function

In [ ]:
# 🧹 Run your existing cleaning function
cleaned_df = clean_tweets('/content/drive/My Drive/_thesis_dogwhistles/Clean-tweets/left-new-french-tweets.csv', '/content/drive/My Drive/_thesis_dogwhistles/Clean-tweets/left-new-cleaned-french-tweets.csv') # output file will still be in /content

Remaining rows: 73363
Total number of tokens: 1478776


In [ ]:
# inspect random samples

import pandas as pd

# Load the final cleaned file
df = pd.read_csv('/content/drive/My Drive/_thesis_dogwhistles/Clean-tweets/left-new-cleaned-french-tweets.csv', dtype={'id': str})

# Check basic info
print(f"Loaded {df.shape[0]} tweets.")
print("Column names:", df.columns.tolist())

# Show 5 random tweets
df.sample(5, random_state=645)

Loaded 73363 tweets.
Column names: ['id', 'username', 'content', 'created_at', 'has_photo', 'has_video', 'is_response']


,id,username,content,created_at,has_photo,has_video,is_response
40520,1922360567337017841,naovic,pourquoi tu la suit t’es masochiste. et en plu...,2025-05-13 18:37:29+00:00,False,False,True
7620,1914032513820787174,sbr_colin,si moueffek pouvait être régulier. si seulemen...,2025-04-20 19:04:46+00:00,False,False,False
46860,1808918253340590258,david73490,vous nagez vraiment en plein délire,2024-07-04 17:38:14+00:00,False,False,True
18542,1921146354799087666,brion_isabelle,comment va carotte ? et comment va son binôme ...,2025-05-10 10:12:38+00:00,False,False,True
30469,1845577278652096856,danielladu35,l’hypocrisie de ce mode de vie marketé sur les...,2024-10-13 21:28:07+00:00,False,False,False
